In [ ]:

!pip install google-cloud-media-translation
!pip install moviepy
!pip install google-cloud-speech==2.19.0
!pip install google-cloud-language
!pip install google-cloud-translate





In [1]:
import io
import os
from google.cloud import mediatranslation
from google.protobuf import duration_pb2
from google.cloud import speech_v1 as speech
from google.cloud import mediatranslation, language_v1
from google.cloud import translate_v2 as translate

# Set Google Cloud credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'creds.json'

In [2]:
# Set input and output file paths
video_file = 'myvideo.mp4'
with open('output.txt', 'w') as f:
    f.write('This is the output file.')
output_file = 'output.txt'



In [3]:
# Extract audio from video and write to temporary audio file
import moviepy.editor as mp
video = mp.VideoFileClip(video_file)
audio = video.audio
audio.write_audiofile("temp_audio_file.wav", fps=16000, nbytes=2, codec='pcm_s16le')


MoviePy - Writing audio in temp_audio_file.wav


MoviePy - Done.


In [4]:

def translate_from_file(input_file_path="temp_audio_file.wav", output_file_path="output.txt"):

    client = mediatranslation.SpeechTranslationServiceClient()
    language_client = language_v1.LanguageServiceClient()

    # The `sample_rate_hertz` field is not required for FLAC and WAV (Linear16)
    # encoded data. Other audio encodings must provide the sampling rate.
    audio_config = mediatranslation.TranslateSpeechConfig(
        audio_encoding="linear16",
        source_language_code="hi-IN",
        target_language_code="en-US",
    )

    streaming_config = mediatranslation.StreamingTranslateSpeechConfig(
        audio_config=audio_config, single_utterance=True
    )

    def request_generator(config, audio_file_path):

        # The first request contains the configuration.
        # Note that audio_content is explicitly set to None.
        yield mediatranslation.StreamingTranslateSpeechRequest(streaming_config=config)

        with open(audio_file_path, "rb") as audio:
            while True:
                chunk = audio.read(4096)
                if not chunk:
                    break
                yield mediatranslation.StreamingTranslateSpeechRequest(
                    audio_content=chunk
                )

    requests = request_generator(streaming_config, input_file_path)
    responses = client.streaming_translate_speech(requests)

    with io.open(output_file_path, "w", encoding="utf-8") as f:
        for response in responses:
            # Once the transcription settles, the response contains the
            # is_final result. The other results will be for subsequent portions of
            # the audio.
            result = response.result
            translation = result.text_translation_result.translation

            if result.text_translation_result.is_final:
                f.write("Final translation: {0}\n".format(translation))
                document = language_v1.Document(content=translation, type_=language_v1.Document.Type.PLAIN_TEXT)
                response = language_client.analyze_entities(document=document, encoding_type=language_v1.EncodingType.UTF8)
                f.write("Entities:\n")
                for entity in response.entities:
                    f.write(f"\tName: {entity.name}, Type: {entity.type_.name}\n")
                break

            f.write("Partial translation: {0}\n".format(translation))
if __name__ == '__main__':
    translate_from_file()

In [5]:

def translate_file(input_file_path="output.txt", output_file_path="translated_output.txt"):
    # Instantiates a client
    client = translate.Client()

    with io.open(input_file_path, "r", encoding="utf-8") as f:
        text = f.read()

    # Translates the text
    translation = client.translate(text, target_language="hi")

    with io.open(output_file_path, "w", encoding="utf-8") as f:
        f.write(translation["translatedText"])

if __name__ == '__main__':
    translate_file()

In [6]:
# Delete temporary audio file
import os
os.remove("temp_audio_file.wav")